In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
A220_tech_docs_text = dataiku.Folder("rhnW9xGx")
A220_tech_docs_text_info = A220_tech_docs_text.get_info()


# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

corpus_strings_statistics_with_pareto_df = ... # Compute a Pandas dataframe to write into corpus_strings_statistics_with_pareto


# Write recipe outputs
corpus_strings_statistics_with_pareto = dataiku.Dataset("corpus_strings_statistics_with_pareto")
corpus_strings_statistics_with_pareto.write_with_schema(corpus_strings_statistics_with_pareto_df)

In [0]:
import dataiku
import pandas as pd
import numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
A220_tech_docs_text = dataiku.Folder("rhnW9xGx")
A220_tech_docs_text_info = A220_tech_docs_text.get_info()

In [0]:
# Assuming we have a function to read the corpus from the folder
def read_corpus(folder):
    # Placeholder for actual implementation
    # This function should return a list of strings from the corpus
    return ["example", "text", "for", "corpus", "example", "text"]

corpus = read_corpus(A220_tech_docs_text)

# Compute frequency of each string in the corpus
frequency = pd.Series(corpus).value_counts().reset_index()
frequency.columns = ['string', 'frequency']

# Calculate relative frequency
total_count = frequency['frequency'].sum()
frequency['relative_frequency'] = frequency['frequency'] / total_count

# Rank the strings by frequency
frequency['rank'] = frequency['frequency'].rank(method='min', ascending=False)

# Sort by string in ascending order before calculating cumulative sum
frequency = frequency.sort_values(by='string')

# Calculate cumulative sum of frequencies
frequency['cumulative_sum'] = frequency['frequency'].cumsum()

# Compute recipe outputs
corpus_strings_statistics_with_pareto_df = frequency

# Write recipe outputs
corpus_strings_statistics_with_pareto = dataiku.Dataset("corpus_strings_statistics_with_pareto")
corpus_strings_statistics_with_pareto.write_with_schema(corpus_strings_statistics_with_pareto_df)
